<a href="https://colab.research.google.com/github/mgp87/Jupyter_Notebooks_Collection/blob/main/CoTCog_1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

class ChainOfThoughtReasoningModel(nn.Module):

    def __init__(self, num_layers=10, hidden_size=448, attention_heads=24, vocab_size=10000, num_classes=2, dropout_prob=0.2):
        super().__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, hidden_size)

        # Positional encoding
        self.positional_encoding = nn.Parameter(torch.randn(1, 10000, hidden_size))

        # Bidirectional transformer encoder
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(
                d_model=hidden_size,
                nhead=attention_heads,
                dim_feedforward=hidden_size * 4,
                dropout=dropout_prob
            ),
            num_layers=num_layers,
            norm=nn.LayerNorm(hidden_size),
        )

        # Gated fusion mechanism
        self.gated_fusion_layer = nn.Linear(hidden_size * 2, hidden_size)

        # Recurrent attention layer
        self.recurrent_attention = nn.GRU(hidden_size, hidden_size)

        # Output layer with dropout
        self.output_layer = nn.Sequential(
            nn.LayerNorm(hidden_size),
            nn.Dropout(p=dropout_prob),
            nn.Linear(hidden_size, num_classes)
        )

        # Initialize the weights of the embedding layer
        torch.nn.init.normal_(self.embedding.weight, mean=0.0, std=0.01)

    def forward(self, input_ids):
        # Embed the input words and add positional encodings
        embeddings = self.embedding(input_ids) + self.positional_encoding

        # Apply the bidirectional transformer encoder
        outputs = self.transformer_encoder(embeddings)

        # Apply the gated fusion mechanism
        fused_outputs = self.gated_fusion_layer(outputs)

        # Apply the recurrent attention layer
        recurrent_attention_outputs, _ = self.recurrent_attention(fused_outputs)

        # Compute the output
        output = self.output_layer(recurrent_attention_outputs)

        return output

In [ ]:
    def save_to_file(self, file_path):
        """Saves the model to a file so that it can be fine tuned later."""
        torch.save(self.state_dict(), file_path)

In [ ]:
import torch

# Load the saved model
model = ChainOfThoughtReasoningModel()
model.load_state_dict(torch.load('model.pt'))

# Fine tune the model for a new task
# ...

# Save the fine tuned model
model.save_to_file('fine_tuned_model.pt')